In [1]:
from datasets import load_dataset
import pandas as pd
import re

/Users/harshit/Library/Caches/pypoetry/virtualenvs/ai-code-reviewer-e5MqfTei-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_name = "Nan-Do/code-search-net-python"
dataset = load_dataset(dataset_name, split="train") # We'll just use the training split for now

Generating train split: 100%|██████████| 455243/455243 [00:05<00:00, 88944.98 examples/s] 


In [ ]:
# Convert to DataFrame
df = pd.DataFrame(dataset)

# Display the first few rows
df.head()